In [16]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from tensorflow import keras

In [17]:
def append_ext(fn):
    return fn+".png"
def hotOne(fn):
    return fn+".png"
def toString(fn):
    return str(fn)

df = pd.read_csv('../input/vcomlabels/multiclass.csv')
df["id"]=df["id"].apply(append_ext)
df["attribute_ids"] = df["attribute_ids"].apply(toString)

"""temp = to_categorical(np.array(df["attribute_ids"]))
df["attribute_ids"] = pd.Series(temp.tolist())
"""
df

,id,attribute_ids
0,1000fe2e667721fe.png,51
1,10041eb49b297c08.png,51
2,100501c227f8beea.png,13
3,1008abd71f3ed5bc.png,70
4,100a0dcde728cb36.png,51
...,...,...
22082,ffec5791dc172baf.png,13
22083,ffed849cb177f26c.png,13
22084,fff30f4d700df04e.png,25
22085,fff600d362e3e868.png,13


In [18]:
unique, counts = np.unique(df["attribute_ids"], return_counts=True)
counts = dict(zip(unique, counts))
counts

{'0': 18,
 '1': 100,
 '10': 10,
 '12': 14,
 '13': 9151,
 '14': 408,
 '15': 34,
 '16': 12,
 '17': 20,
 '18': 121,
 '2': 14,
 '22': 22,
 '23': 244,
 '24': 41,
 '25': 676,
 '26': 177,
 '27': 13,
 '28': 12,
 '29': 530,
 '32': 72,
 '33': 292,
 '35': 119,
 '38': 10,
 '40': 48,
 '41': 141,
 '42': 50,
 '43': 74,
 '44': 35,
 '45': 514,
 '46': 11,
 '48': 34,
 '49': 12,
 '50': 29,
 '51': 7463,
 '52': 17,
 '54': 10,
 '55': 91,
 '56': 10,
 '58': 240,
 '59': 22,
 '60': 74,
 '61': 71,
 '62': 11,
 '64': 13,
 '65': 72,
 '66': 11,
 '67': 22,
 '69': 43,
 '70': 831,
 '9': 28}

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['id'], df['attribute_ids'], stratify = df['attribute_ids'], test_size=0.15, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, stratify = y_train, test_size=0.15, random_state=42)
X_train,X_test

(17230    cbe0f4556cb5926b.png
 34       1055abe56bbddf45.png
 17561    cfc3bf7f7143a082.png
 1193     1c7c0c652d4e0bfe.png
 12755    9b33c57a80667977.png
                  ...         
 19839    e8612a41bce5eeb2.png
 20906    f383ed48af7f5d36.png
 17151    cb0d737656d1a3b4.png
 3212     329dd67046b33db7.png
 11965     9278ff0a708e617.png
 Name: id, Length: 15957, dtype: object,
 18120    d60fc61f82c4c07f.png
 14585    aea5f1059186bb49.png
 772      17cbab68101597bf.png
 10782    851faf0a057f66bb.png
 13740    a57d2da847a675cb.png
                  ...         
 10776     8519508edd3b3f9.png
 2423     2a86554e8cd80e35.png
 16726    c6504f3e9875bc2a.png
 20813    f2b19dfdd876b57d.png
 3616     373198796c838f8e.png
 Name: id, Length: 3314, dtype: object)

In [20]:
frame = [X_train, y_train]
numpy_data= np.array(frame)
df_train = pd.DataFrame(data=numpy_data).T.rename(columns={0: "id", 1: "attribute_ids"})
df_train

,id,attribute_ids
0,cbe0f4556cb5926b.png,51
1,1055abe56bbddf45.png,13
2,cfc3bf7f7143a082.png,13
3,1c7c0c652d4e0bfe.png,23
4,9b33c57a80667977.png,33
...,...,...
15952,e8612a41bce5eeb2.png,70
15953,f383ed48af7f5d36.png,18
15954,cb0d737656d1a3b4.png,70
15955,329dd67046b33db7.png,13


In [21]:
frame = [X_val, y_val]
numpy_data= np.array(frame)
df_val = pd.DataFrame(data=numpy_data).T.rename(columns={0: "id", 1: "attribute_ids"})
df_val

,id,attribute_ids
0,35a5ccde8844775f.png,13
1,ea3d944f2a743e84.png,51
2,aeee813d29250811.png,13
3,afe044bc8340b63d.png,60
4,7f6bcd194c24f4c9.png,13
...,...,...
2811,8044ca60db940d5f.png,13
2812,3ddb29fec0a825f5.png,51
2813,c93898b231c16710.png,61
2814,918bfaac3f6948.png,51


In [22]:
frame = [X_test, y_test]
numpy_data= np.array(frame)
df_test = pd.DataFrame(data=numpy_data).T.rename(columns={0: "id", 1: "attribute_ids"})
df_test

,id,attribute_ids
0,d60fc61f82c4c07f.png,13
1,aea5f1059186bb49.png,51
2,17cbab68101597bf.png,13
3,851faf0a057f66bb.png,13
4,a57d2da847a675cb.png,25
...,...,...
3309,8519508edd3b3f9.png,29
3310,2a86554e8cd80e35.png,25
3311,c6504f3e9875bc2a.png,51
3312,f2b19dfdd876b57d.png,13


In [23]:
unique, counts = np.unique(df_train["attribute_ids"], return_counts=True)
counts = dict(zip(unique, counts))
max_key = max(counts, key=counts.get)
max_value = counts[max_key]
target_count = {}
for key in counts.keys():
    val = counts[key]
    if max_value / val < 2:
        target_count[key] = val
    elif max_value / val < 4:
        target_count[key] = val * 3
    elif max_value / val < 8:
        target_count[key] = val * 5
    elif max_value / val < 14:
        target_count[key] = val * 7
    elif max_value / val < 22:
        target_count[key] = val * 9
    elif max_value / val < 32:
        target_count[key] = val * 11
    else:
        target_count[key] = val * 13
target_count

{'0': 169,
 '1': 936,
 '10': 104,
 '12': 130,
 '13': 6611,
 '14': 3245,
 '15': 325,
 '16': 104,
 '17': 182,
 '18': 1144,
 '2': 130,
 '22': 208,
 '23': 2288,
 '24': 390,
 '25': 3423,
 '26': 1651,
 '27': 117,
 '28': 117,
 '29': 3438,
 '32': 676,
 '33': 2321,
 '35': 1118,
 '38': 104,
 '40': 455,
 '41': 1326,
 '42': 468,
 '43': 702,
 '44': 325,
 '45': 3339,
 '46': 104,
 '48': 325,
 '49': 104,
 '50': 273,
 '51': 5392,
 '52': 156,
 '54': 104,
 '55': 845,
 '56': 104,
 '58': 2249,
 '59': 208,
 '60': 702,
 '61': 663,
 '62': 104,
 '64': 117,
 '65': 676,
 '66': 104,
 '67': 208,
 '69': 403,
 '70': 4200,
 '9': 260}

In [24]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=42,sampling_strategy=target_count)
X = np.array(df_train["id"]).reshape(-1, 1)
y = np.array(df_train["attribute_ids"]).reshape(-1, 1)
X_res, y_res = ros.fit_resample(X, y)


In [25]:
frame = [X_res[:,0], y_res]
numpy_data= np.array(frame)
df_train = pd.DataFrame(data=numpy_data).T.rename(columns={0: "id", 1: "attribute_ids"})
df_train = df_train.sample(frac=1).reset_index(drop=True)
df_train

,id,attribute_ids
0,7783f40021f6a19a.png,43
1,2a100c0505d3e0f8.png,45
2,f3000fa15b3f04f4.png,45
3,272d752c86c37919.png,29
4,167589218dc0580e.png,40
...,...,...
52842,2afb59f3598cd2f5.png,15
52843,e883b5fb629dae14.png,26
52844,424d68555f06e276.png,43
52845,6dba0654aa23663e.png,23


In [26]:
def process(image):
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    return image

datagen=ImageDataGenerator(rescale=1./255, validation_split=0.2, preprocessing_function=process,
                            rotation_range=30,
                            width_shift_range=0,
                            height_shift_range=0.2,
                            zoom_range=0.2,
                            horizontal_flip=True,
                          )

datagen_test=ImageDataGenerator(rescale=1./255, validation_split=0.0, preprocessing_function=process,
                            rotation_range=30,
                            width_shift_range=0,
                            height_shift_range=0,
                            zoom_range=0,
                            horizontal_flip=False,
                          )

train_generator=datagen.flow_from_dataframe(dataframe=df_train, directory="../input/imet-2019-fgvc6/train",
                                            x_col="id", y_col="attribute_ids", class_mode="categorical",
                                            target_size=(224,224), batch_size=16,subset="training")
validation_generator=datagen_test.flow_from_dataframe(dataframe=df_val, directory="../input/imet-2019-fgvc6/train",
                                            x_col="id", y_col="attribute_ids", class_mode="categorical",
                                            target_size=(224,224), batch_size=16,subset="training")

test_generator=datagen_test.flow_from_dataframe(dataframe=df_test, directory="../input/imet-2019-fgvc6/train",
                                            x_col="id", y_col="attribute_ids", class_mode="categorical",
                                            target_size=(224,224), batch_size=16,subset="training")

Found 42278 validated image filenames belonging to 50 classes.
Found 2816 validated image filenames belonging to 50 classes.
Found 3314 validated image filenames belonging to 50 classes.


In [27]:
#train_generator.class_indices,validation_generator.class_indices

In [28]:
key_list = list(train_generator.class_indices.keys())
val_list = list(train_generator.class_indices.values())
def get_label(index):
    position = val_list.index(index)
    return key_list[position]
print(get_label(4))

13


In [29]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_prc', 
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)


In [ ]:
from sklearn.utils import class_weight 
class_weights = class_weight.compute_class_weight(
           'balanced',
            np.unique(train_generator.classes), 
            train_generator.classes)

train_class_weights = dict(enumerate(class_weights))
train_class_weights

In [31]:
unique, counts = np.unique(df["attribute_ids"], return_counts=True)
counts = dict(zip(unique, counts))
key_list_counts = list(counts.keys())
class_weight = {}
i = 0
for k in key_list_counts:
    class_weight[i] = (1/counts[k]) * (len(df["attribute_ids"])/len(unique))
    i = i + 1
class_weight

{0: 24.54111111111111,
 1: 4.4174,
 2: 44.17400000000001,
 3: 31.552857142857142,
 4: 0.048272319965031146,
 5: 1.0826960784313726,
 6: 12.99235294117647,
 7: 36.81166666666667,
 8: 22.087000000000003,
 9: 3.650743801652893,
 10: 31.552857142857142,
 11: 20.07909090909091,
 12: 1.8104098360655738,
 13: 10.774146341463416,
 14: 0.6534615384615384,
 15: 2.4957062146892657,
 16: 33.980000000000004,
 17: 36.81166666666667,
 18: 0.8334716981132075,
 19: 6.135277777777778,
 20: 1.512808219178082,
 21: 3.7121008403361344,
 22: 44.17400000000001,
 23: 9.202916666666667,
 24: 3.1329078014184395,
 25: 8.8348,
 26: 5.96945945945946,
 27: 12.621142857142857,
 28: 0.8594163424124514,
 29: 40.15818181818182,
 30: 12.99235294117647,
 31: 36.81166666666667,
 32: 15.232413793103449,
 33: 0.05919067399169235,
 34: 25.98470588235294,
 35: 44.17400000000001,
 36: 4.854285714285715,
 37: 44.17400000000001,
 38: 1.8405833333333332,
 39: 20.07909090909091,
 40: 5.96945945945946,
 41: 6.22169014084507,
 42: 4

In [32]:
METRICS = [
      keras.metrics.CategoricalAccuracy(name='acc'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
      'accuracy',
]


In [34]:
from keras.applications.inception_v3 import preprocess_input
from keras.applications.inception_v3 import decode_predictions
from keras.applications.inception_v3 import InceptionV3

In [35]:
#model2 = VGG16(include_top=False,weights="imagenet",input_shape=(64, 64, 3))
model2 = InceptionV3(include_top = False, weights = "imagenet", input_shape=(224, 224, 3))


87916544/87910968 [==============================] - 1s 0us/step


In [ ]:
flat1 = Flatten()(model2.layers[-1].output)
class1 = Dense(1024, activation='relu')(flat1)
dropout1 = Dropout(0.5)(class1)
output = Dense(50, activation='softmax')(dropout1)
# define new model
model2 = Model(inputs=model2.inputs, outputs=output)


In [41]:
from keras.layers import GlobalAveragePooling2D
from keras.layers import GlobalMaxPooling2D
from keras.models import Sequential
base_model = InceptionV3(weights='imagenet', 
                                include_top=False, )
add_model = Sequential()
add_model.add(base_model)
add_model.add(GlobalAveragePooling2D())
add_model.add(Dense(1024, activation='relu'))
add_model.add(Dropout(0.5))
add_model.add(Dense(50, 
                    activation='softmax'))
model2 = add_model
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, None, None, 2048)  21802784  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 50)                51250     
Total params: 23,952,210
Trainable params: 23,917,778
Non-trainable params: 34,432
_________________________________________________________________


In [42]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size


model2.compile( keras.optimizers.RMSprop(learning_rate=0.0001), 
               loss="categorical_crossentropy", metrics=METRICS)
model2.fit(train_generator, steps_per_epoch=STEP_SIZE_TRAIN,
           validation_data=validation_generator, validation_steps=STEP_SIZE_VALID,
           callbacks=[early_stopping],epochs=15, class_weight = train_class_weights)

Epoch 1/15
2642/2642 [==============================] - 973s 364ms/step - loss: 2.4561 - acc: 0.2953 - precision: 0.5410 - recall: 0.1590 - auc: 0.8710 - prc: 0.2820 - accuracy: 0.3285 - val_loss: 2.0745 - val_acc: 0.4840 - val_precision: 0.5265 - val_recall: 0.4411 - val_auc: 0.9338 - val_prc: 0.4945 - val_accuracy: 0.4840
Epoch 2/15
2642/2642 [==============================] - 781s 296ms/step - loss: 0.6243 - acc: 0.6497 - precision: 0.7564 - recall: 0.5673 - auc: 0.9778 - prc: 0.7204 - accuracy: 0.6497 - val_loss: 1.7002 - val_acc: 0.5582 - val_precision: 0.6025 - val_recall: 0.5082 - val_auc: 0.9504 - val_prc: 0.5960 - val_accuracy: 0.5582
Epoch 3/15
2642/2642 [==============================] - 777s 294ms/step - loss: 0.3277 - acc: 0.7578 - precision: 0.8246 - recall: 0.7065 - auc: 0.9869 - prc: 0.8423 - accuracy: 0.7578 - val_loss: 2.4404 - val_acc: 0.5167 - val_precision: 0.5481 - val_recall: 0.4918 - val_auc: 0.9132 - val_prc: 0.5120 - val_accuracy: 0.5167
Epoch 4/15
2642/2642 [

In [ ]:
image = load_img('../input/imet-2019-fgvc6/train/100501c227f8beea.png', target_size=(224, 224))
image = img_to_array(image)
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
image = preprocess_input(image)

In [ ]:
yhat = model2.predict(image)
print(yhat)
y_classes = yhat.argmax(axis=-1)
print(y_classes)
print(get_label(y_classes[0]))

In [ ]:
def predict_img(path):
    image = load_img('../input/imet-2019-fgvc6/train/' + path + '.png', target_size=(224, 224))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    yhat = model2.predict(image)
    print(yhat)
    y_classes = yhat.argmax(axis=-1)
    print(get_label(y_classes[0]))
train_generator.class_indices

In [ ]:
predict_img('1000fe2e667721fe')
predict_img('10041eb49b297c08')
predict_img('100501c227f8beea')
predict_img('1008abd71f3ed5bc')
predict_img('100a0dcde728cb36')
predict_img('100bbf5e832083d3')
predict_img('100d750286e85bf3')
predict_img('100ef61c00e1b5d3')
predict_img('1012a188a1fd6166')

In [43]:
test_generator.reset()
evaluate = model2.evaluate(test_generator)
evaluate

208/208 [==============================] - 91s 437ms/step - loss: 2.8431 - acc: 0.6880 - precision: 0.6978 - recall: 0.6765 - auc: 0.9309 - prc: 0.6805 - accuracy: 0.6880


[2.8430898189544678,
 0.6879903674125671,
 0.6977902054786682,
 0.6765238642692566,
 0.9308665990829468,
 0.6804525852203369,
 0.6879903674125671]

In [ ]:
model2.metrics_names

In [44]:
from sklearn.metrics import classification_report, confusion_matrix
test_generator.reset()
Y_pred = model2.predict(test_generator)
y_pred = np.argmax(Y_pred, axis=1)

In [46]:
target_names= list(target_count.keys())

In [48]:
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))
print('Classification Report')
#target_names = ['Cats', 'Dogs', 'Horse']
print(classification_report(test_generator.classes, y_pred, target_names = target_names))

Confusion Matrix
[[0 0 0 ... 0 1 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 5 0]
 [0 0 0 ... 0 0 0]]
Classification Report
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           1       0.00      0.00      0.00        15
          10       0.00      0.00      0.00         1
          12       0.00      0.00      0.00         2
          13       0.42      0.34      0.38      1373
          14       0.01      0.02      0.01        61
          15       0.00      0.00      0.00         5
          16       0.00      0.00      0.00         2
          17       0.00      0.00      0.00         3
          18       0.00      0.00      0.00        18
           2       0.00      0.00      0.00         2
          22       0.00      0.00      0.00         3
          23       0.00      0.00      0.00        37
          24       0.00      0.00      0.00         6
          25       0.03      0

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
